# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [72]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [73]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [74]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [149]:
total_customer_purchase = data.groupby(by=['CustomerID','ProductID','ProductName']).sum().reset_index(drop=False)
total_customer_purchase.head()

,CustomerID,ProductID,ProductName,SalesID,Quantity
0,33,23,Crab - Imitation Flakes,6515659,1
1,33,51,Wine - Crozes Hermitage E.,5332057,1
2,33,53,Cassis,1888258,1
3,33,67,Wine - Sogrape Mateus Rose,3975703,1
4,33,70,Squid - Tubes / Tenticles 10/20,2235546,1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [150]:
recommend = pd.pivot_table(total_customer_purchase, values='Quantity', index='ProductName', columns='CustomerID').fillna(0)
recommend.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [204]:
distances = pdist(recommend.T,'cosine')
distances_matrix = pd.DataFrame(squareform(distances),columns=recommend.columns, index=recommend.columns)
display(distances_matrix)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,0.886781,0.774648,0.839772,0.843681,0.848243,0.915650,0.884009,0.875835,0.868135,...,0.885703,0.930549,0.895154,0.955144,0.872026,0.946249,0.891093,0.922307,0.888643,0.897020
200,0.886781,0.000000,0.861622,0.868815,0.918555,0.926067,0.795510,0.935250,0.864456,0.846889,...,0.834109,0.875937,0.801003,0.839743,0.885697,0.915985,0.744661,0.861214,0.900539,0.896524
264,0.774648,0.861622,0.000000,0.879829,0.817627,0.903428,0.855401,0.942005,0.862039,0.907695,...,0.828554,0.902768,0.868942,0.880384,0.808039,0.784998,0.823027,0.857563,0.802031,0.897020
356,0.839772,0.868815,0.879829,0.000000,0.901204,0.908448,0.862918,0.848803,0.934606,0.924994,...,0.864555,0.920991,0.875755,0.872427,0.890811,0.885348,0.922566,0.864967,0.835782,0.865764
412,0.843681,0.918555,0.817627,0.901204,0.000000,0.872402,0.843972,0.825672,0.885161,0.841452,...,0.894287,0.768755,0.890907,0.889368,0.846130,0.863288,0.924455,0.856285,0.828344,0.809510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.946249,0.915985,0.784998,0.885348,0.863288,0.842051,0.850545,0.875503,0.868375,0.899353,...,0.877321,0.854225,0.912473,0.843081,0.755807,0.000000,0.779202,0.839396,0.881952,0.926312
98069,0.891093,0.744661,0.823027,0.922566,0.924455,0.919993,0.871885,0.901903,0.879989,0.847056,...,0.889526,0.865745,0.885994,0.927741,0.802092,0.779202,0.000000,0.824781,0.844531,0.863140
98159,0.922307,0.861214,0.857563,0.864967,0.856285,0.873163,0.867060,0.866702,0.835112,0.866645,...,0.842378,0.795676,0.867460,0.917521,0.847048,0.839396,0.824781,0.000000,0.840744,0.881655


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [152]:
def similar(customer):
    return distances_matrix[customer].sort_values()[1:]
similar(412)[:5]

CustomerID
3317    10.677078
3535    10.770330
1008    11.000000
1072    11.090537
639     11.135529
Name: 412, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [153]:
def purchaseSimilar(customer):
    return total_customer_purchase[total_customer_purchase['CustomerID'].isin(list(similar(customer)[:5].index))]
purchase_similar_33 = purchaseSimilar(33)
purchase_similar_33.head()

,CustomerID,ProductID,ProductName,SalesID,Quantity
131,264,2,Cookie Chocolate Chip With,928142,1
132,264,30,"Cheese - Brie, Triple Creme",6213860,1
133,264,48,Pecan Raisin - Tarts,3595141,1
134,264,49,Bacardi Breezer - Tropical,756802,1
135,264,59,Pastry - Butterscotch Baked,6696158,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [154]:
def recommend_cust(customer): 
    return purchaseSimilar(customer).groupby(by='ProductName').sum().sort_values(by='Quantity',ascending=False)
recommend_33 = recommend_cust(33)
recommend_33

,CustomerID,ProductID,SalesID,Quantity
ProductName,,,,
Butter - Unsalted,5820,804,13554503,3
Wine - Ej Gallo Sierra Valley,6622,884,5892505,3
Towels - Paper / Kraft,7104,957,5618699,3
Soup - Campbells Bean Medley,7116,1146,9218997,3
Wine - Blue Nun Qualitatswein,7104,807,9241577,3
...,...,...,...,...
Hinge W Undercut,264,297,6577594,1
Ice Cream Bar - Hageen Daz To,2503,246,5840790,1
Jagermeister,3535,374,5687769,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [184]:
def recommendation(customer):
    return list(recommend_cust(customer).filter(items=(recommend[customer] == 0).index).index)
recommend_33_top_5 = recommendation(33)[:5]
recommend_33_top_5

['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Towels - Paper / Kraft',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [185]:
recommendations = {}
customers = list(distances_matrix.columns)
for cust in customers:
    recommendations[cust] = recommendation(cust)[:5]

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [201]:
recommendations_df = pd.DataFrame(recommendations)
recommendations_df.iloc[:,0:5]

,33,200,264,356,412
0,Butter - Unsalted,Soup - Campbells Bean Medley,Soupfoamcont12oz 112con,"Veal - Inside, Choice",Olive - Spread Tapenade
1,Wine - Ej Gallo Sierra Valley,Muffin - Carrot Individual Wrap,Wine - Two Oceans Cabernet,Wine - Ej Gallo Sierra Valley,Sprouts - Baby Pea Tendrils
2,Towels - Paper / Kraft,Bay Leaf,Bread - Italian Roll With Herbs,Lamb - Ground,Wine - Blue Nun Qualitatswein
3,Soup - Campbells Bean Medley,Pork - Kidney,"Veal - Inside, Choice",Wine - Blue Nun Qualitatswein,"Veal - Inside, Choice"
4,Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole",Potatoes - Idaho 100 Count,Pomello,"Pepper - Black, Whole"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [205]:
recommendations_correlation = {}
customers = list(distances_matrix.columns)
for cust in customers:
    recommendations_correlation[cust] = recommendation(cust)[:5]
recommendations_correlation_df = pd.DataFrame(recommendations_correlation)
recommendations_correlation_df.iloc[:,0:5]

,33,200,264,356,412
0,Knife Plastic - White,Olives - Kalamata,"Lemonade - Natural, 591 Ml",Tomatoes Tear Drop,Banana - Leaves
1,Phyllo Dough,Mustard Prepared,Pickerel - Fillets,Sobe - Tropical Energy,"Salsify, Organic"
2,Cake - Box Window 10x10x2.5,Quiche Assorted,Muffin Mix - Blueberry,Bread - English Muffin,Beans - Wax
3,"Soup - Campbells, Beef Barley",Pop Shoppe Cream Soda,Wine - Blue Nun Qualitatswein,"Thyme - Lemon, Fresh",Durian Fruit
4,Onions - Cippolini,Longos - Grilled Salmon With Bbq,"Water - Mineral, Natural",Olive - Spread Tapenade,Whmis - Spray Bottle Trigger
